# [Updated version of Nicholas Renotte's solution]

In [ ]:
%pip install tensorflow == 2.12 opencv-python mediapipe sklearn matplotlib


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
# import time
from matplotlib import pyplot as plt


In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


In [ ]:
def mediapipe_detection(image: cv2.Mat, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )


In [ ]:
cap = cv2.VideoCapture(r"downloads/PXL_20211114_112209361_compact.mp4")
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
def extract_landmarks(results) -> np.ndarray:
    """Transforms the results from a mediapipe process to a NumPy Array

    Args:
        results: Results from a mediapipe process

    Returns:
        np.ndarray: Vectorized results, missing landmarks are representend as numpy.nan
        
        results.shape = (4,)

        results[0].shape = (468, 3), results[1].shape = (33, 3),
        results[2].shape = (21, 3), results[3].shape = (21, 3)
    """
    face_landmarks = np.zeros((468, 3))
    face_landmarks.fill(np.nan)
    if results.face_landmarks != None:
        for i, landmark in enumerate(results.face_landmarks.landmark):
            face_landmarks[i] = landmark.x, landmark.y, landmark.z
    else:
        face_landmarks.fill(np.nan)

    pose_landmarks = np.zeros((33, 3))
    pose_landmarks.fill(np.nan)
    if results.pose_landmarks != None:
        for i, landmark in enumerate(results.pose_landmarks.landmark):
            pose_landmarks[i] = landmark.x, landmark.y, landmark.z
    else:
        pose_landmarks.fill(np.nan)

    left_hand_landmarks = np.zeros((21, 3))
    left_hand_landmarks.fill(np.nan)
    if results.left_hand_landmarks != None:
        for i, landmark in enumerate(results.left_hand_landmarks.landmark):
            left_hand_landmarks[i] = landmark.x, landmark.y, landmark.z
    else:
        left_hand_landmarks.fill(np.nan)

    right_hand_landmarks = np.zeros((21, 3))
    right_hand_landmarks.fill(np.nan)
    if results.right_hand_landmarks != None:
        for i, landmark in enumerate(results.right_hand_landmarks.landmark):
            right_hand_landmarks[i] = landmark.x, landmark.y, landmark.z
    else:
        right_hand_landmarks.fill(np.nan)

    return np.array([face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks], dtype=object)

=== stopped at 40:08 ===